# Installazione e import pacchetti

In [38]:
# %pip install pandas
# %pip install openpyxl
%pip install statsmodels

     ---------------------------------------- 9.1/9.1 MB 11.2 MB/s eta 0:00:00
     ------------------------------------- 233.7/233.7 KB 14.0 MB/s eta 0:00:00
     --------------------------------------- 36.9/36.9 MB 10.6 MB/s eta 0:00:00


ERROR: Could not install packages due to an OSError: [WinError 32] Impossibile accedere al file. Il file è utilizzato da un altro processo: 'c:\\Users\\marco\\AppData\\Local\\Programs\\Python\\Python310\\Lib\\site-packages\\statsmodels\\stats\\multivariate_tools.py'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'c:\Users\marco\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


  Using cached statsmodels-0.13.2-cp310-cp310-win_amd64.whl (9.1 MB)
  Using cached patsy-0.5.2-py2.py3-none-any.whl (233 kB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\marco\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [39]:
import pandas as pd
from openpyxl import Workbook
import numpy as np
import requests
import io
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS

# Import dataset

In [46]:
url_remo = 'C:/Users/marco/OneDrive/UNIMIB_DataScience/99-PROJECTS/FinancialMarketAnalytics2022/Financial_Market_Analytics/Dataset/NASDAQ_INDEX.xlsx'
df_index = pd.read_excel(url_remo, header=0, parse_dates=[0], index_col=0 )

In [24]:
url_remo= 'C:/Users/marco/OneDrive/UNIMIB_DataScience/99-PROJECTS/FinancialMarketAnalytics2022/Financial_Market_Analytics/Dataset/STOCKS.xlsx'
df_stocks = pd.read_excel(url_remo, header=0, parse_dates=[0], index_col=0 )

In [47]:
display(df_index)

,NASDAQ_Index
Date,
2017-07-07,5656.47
2017-07-10,5694.15
2017-07-11,5709.80
2017-07-12,5778.95
2017-07-13,5793.36
...,...
2022-07-05,11779.90
2022-07-06,11852.59
2022-07-07,12109.05


In [25]:
display(df_stocks)

,"Airbnb, Inc. Class A (ABNB-US)","Lucid Group, Inc. (LCID-US)",Datadog Inc Class A (DDOG-US),"CrowdStrike Holdings, Inc. Class A (CRWD-USA)","Zscaler, Inc. (ZS-US)",Constellation Energy Corporation (CEG-US),"Fortinet, Inc. (FTNT-US)","Palo Alto Networks, Inc. (PANW-US)","Old Dominion Freight Line, Inc. (ODFL-US)",Astrazeneca PLC Sponsored ADR (AZN-US),...,NortonLifeLock Inc. (NLOK-US),"Henry Schein, Inc. (HSIC-US)","Illumina, Inc. (ILMN-US)","Analog Devices, Inc. (ADI-US)","Autodesk, Inc. (ADSK-US)","Mattel, Inc. (MAT-US)",Walgreens Boots Alliance Inc (WBA-US),Vodafone Group Plc Sponsored ADR (VOD-US),Fastenal Company (FAST-US),Cintas Corporation (CTAS-US)
Date,,,,,,,,,,,,,,,,,,,,,
2022-07-08,67.272476,202.123357,288.388815,328.241379,505.727273,142.023810,814.497230,385.031216,418.571115,198.459259,...,NaN,NaN,114.484922,191.079393,174.051491,NaN,48.470376,NaN,230.758715,302.124792
2022-07-07,67.376132,200.303337,286.817577,327.931034,507.363636,140.738095,800.467612,379.911862,419.598302,198.281481,...,NaN,NaN,117.483005,189.333503,176.393728,NaN,49.322318,NaN,231.442242,301.982082
2022-07-06,64.183539,184.226491,287.696405,320.551724,496.212121,135.142857,777.734436,378.229893,405.033708,197.007407,...,NaN,NaN,113.526233,184.401682,171.051103,NaN,48.767265,NaN,229.756209,301.371601
2022-07-05,66.298113,174.216380,288.628495,322.448276,503.969697,130.809524,766.432800,382.769005,397.107502,194.103704,...,NaN,NaN,112.149207,182.324455,171.215641,NaN,49.709565,NaN,226.338574,297.597717
2022-07-01,63.167715,171.890799,269.107856,309.051724,470.818182,137.142857,745.258470,373.301506,400.158401,195.407407,...,NaN,NaN,110.998780,182.120556,168.273326,NaN,49.774106,NaN,226.657553,298.668041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-07-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,106.863780,99.775763,100.517111,102.293870,103.629501,99.615569,100.697044,101.324965,98.359535,100.269563
2017-07-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,105.315030,100.180492,100.650747,102.421308,103.252033,100.336377,100.064541,100.767085,97.334245,100.816618
2017-07-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,102.111932,98.818640,100.604265,101.771378,100.948509,100.336377,99.767652,98.919107,99.316473,99.762150


# Log-Return

In [42]:
df_index_logret=df_index.copy()
df_index_logret['NASDAQ_Index']=df_index_logret['NASDAQ_Index'].pct_change().apply(lambda x: np.log(1+x))
display(df_index_logret)

,NASDAQ_Index
Date,
2017-07-31,NaN
2017-08-01,0.002520
2017-08-02,0.003228
2017-08-03,-0.003902
2017-08-04,0.001477
...,...
2022-07-01,0.007099
2022-07-05,0.016625
2022-07-06,0.006152


In [28]:
df_stocks_retlog = df_stocks.copy()
for column in df_stocks_retlog:
    df_stocks_retlog[column]=df_stocks_retlog[column].pct_change().apply(lambda x: np.log(1+x))
display(df_stocks_retlog)

,"Airbnb, Inc. Class A (ABNB-US)","Lucid Group, Inc. (LCID-US)",Datadog Inc Class A (DDOG-US),"CrowdStrike Holdings, Inc. Class A (CRWD-USA)","Zscaler, Inc. (ZS-US)",Constellation Energy Corporation (CEG-US),"Fortinet, Inc. (FTNT-US)","Palo Alto Networks, Inc. (PANW-US)","Old Dominion Freight Line, Inc. (ODFL-US)",Astrazeneca PLC Sponsored ADR (AZN-US),...,NortonLifeLock Inc. (NLOK-US),"Henry Schein, Inc. (HSIC-US)","Illumina, Inc. (ILMN-US)","Analog Devices, Inc. (ADI-US)","Autodesk, Inc. (ADSK-US)","Mattel, Inc. (MAT-US)",Walgreens Boots Alliance Inc (WBA-US),Vodafone Group Plc Sponsored ADR (VOD-US),Fastenal Company (FAST-US),Cintas Corporation (CTAS-US)
Date,,,,,,,,,,,,,,,,,,,,,
2022-07-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-07-07,0.001540,-0.009045,-0.005463,-0.000946,0.003230,-0.009094,-0.017375,-0.013385,0.002451,-0.000896,...,NaN,NaN,0.025851,-0.009179,0.013367,NaN,0.017424,NaN,0.002958,-0.000472
2022-07-06,-0.048544,-0.083667,0.003059,-0.022760,-0.022224,-0.040568,-0.028811,-0.004437,-0.035328,-0.006446,...,NaN,NaN,-0.034260,-0.026394,-0.030756,NaN,-0.011317,NaN,-0.007312,-0.002024
2022-07-05,0.032415,-0.055868,0.003235,0.005899,0.015513,-0.032590,-0.014638,0.011929,-0.019763,-0.014849,...,NaN,NaN,-0.012204,-0.011329,0.000961,NaN,0.019138,NaN,-0.014987,-0.012601
2022-07-01,-0.048368,-0.013439,-0.070028,-0.042434,-0.068044,0.047281,-0.028016,-0.025045,0.007653,0.006694,...,NaN,NaN,-0.010311,-0.001119,-0.017334,NaN,0.001298,NaN,0.001408,0.003590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-07-13,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.009265,-0.011391,0.003880,-0.015820,-0.015477,-0.015794,-0.011471,-0.001376,0.006740,0.000316
2017-07-12,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.014599,0.004048,0.001329,0.001245,-0.003649,0.007210,-0.006301,-0.005521,-0.010479,0.005441
2017-07-11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.030887,-0.013687,-0.000462,-0.006366,-0.022562,0.000000,-0.002971,-0.018509,0.020161,-0.010514


# Rolling regression

In [43]:
endog = df_stocks_retlog['Cintas Corporation (CTAS-US)']
exog = df_index_logret['NASDAQ_Index']

In [45]:
rols = RollingOLS(endog, exog, window=180)

ValueError: operands could not be broadcast together with shapes (1260,1) (1244,1) 